# lettuce size Prediction with expected days

In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

In [48]:
#plants sheet 
df = pd.read_excel(r'plants.xlsx') 
df.head(5)

,Batch Number,Plant Date,Class,Fresh Weight (g),Head Weight (g),Radial Diameter (mm),Polar Diameter (mm),Diameter Ratio,Leaves,Density (kg/L),Leaf Area (cm^2),Square ID,Check Date,Flight Date,Remove
0,402,2020-04-11,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,185.0,208,2020-04-30,NaT,NaN
1,402,2020-04-11,4,15.0,NaN,NaN,NaN,NaN,11.0,NaN,156.0,208,2020-04-30,NaT,NaN
2,402,2020-04-11,1,16.0,NaN,NaN,NaN,NaN,12.0,NaN,137.0,208,2020-04-30,NaT,NaN
3,402,2020-04-11,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,225.0,208,2020-04-30,NaT,NaN
4,402,2020-04-11,3,15.0,NaN,NaN,NaN,NaN,11.0,NaN,157.0,208,2020-04-30,NaT,NaN


In [49]:
#flight_dates sheet 
df1 = pd.read_excel(r'flight_dates.xlsx') 
df1.head(5)

,Batch Number,Flight Date
0,402,2020-04-30
1,408,2020-04-30
2,409,2020-04-30
3,466,2020-07-01
4,467,2020-07-01


In [50]:
#weather sheet 
df2 = pd.read_excel(r'weather.xlsx') 
df2.head(5)

,Flight Date,Solar Radiation [avg],Precipitation [sum],Wind Speed [avg],Wind Speed [max],Battery Voltage [last],Leaf Wetness [time],Air Temperature [avg],Air Temperature [max],Air Temperature [min],Relative Humidity [avg],Dew Point [avg],Dew Point [min],ET0 [result]
0,2014-05-14,159,0.0,1.4,3.3,6834,70,14.27,18.90,7.19,64.95,7.0,5.6,NaN
1,2014-05-15,280,0.0,0.6,2.3,6894,500,13.51,22.06,3.04,72.91,7.6,3.0,3.6
2,2014-05-16,186,3.4,0.8,2.7,6874,375,15.65,24.24,7.23,74.69,10.5,6.5,2.9
3,2014-05-17,221,0.0,1.0,3.3,6874,565,17.46,25.70,9.93,71.42,11.5,8.1,3.6
4,2014-05-18,255,0.0,2.1,5.5,6854,625,16.85,24.62,8.45,74.45,11.4,8.3,4.2


In [51]:
#planting sheet 
df3 = pd.read_excel(r'planting.xlsx') 
df3.head(5)

,Crop,Column2,Column3,Plant Date,Column1,Region,Column4,Planting_Batch,Volume_Planted,Planting_Wk
0,3,NaN,NaN,2020-02-06,NaN,0.0,NaN,392,198912.0,6.0
1,3,NaN,NaN,2020-02-07,NaN,0.0,NaN,503,63840.0,6.0
2,3,NaN,NaN,2020-02-07,NaN,0.0,NaN,614,179928.0,6.0
3,3,NaN,NaN,2020-02-08,NaN,0.0,NaN,725,200256.0,6.0
4,3,NaN,NaN,2020-02-10,NaN,0.0,NaN,770,13944.0,7.0


Add flight dates from 2020 planting data

In [52]:
df3 = pd.read_excel('flight_dates.xlsx', usecols=['Flight Date'])
df.loc[:, ['Flight Date']] = df3[['Flight Date']]

Add missing planting dates from 2020 planting data 

In [53]:
df4 = pd.read_excel('planting.xlsx', usecols=['Plant Date'])
df.loc[:, ['Plant Date']] = df4[['Plant Date']]

In [54]:
df

,Batch Number,Plant Date,Class,Fresh Weight (g),Head Weight (g),Radial Diameter (mm),Polar Diameter (mm),Diameter Ratio,Leaves,Density (kg/L),Leaf Area (cm^2),Square ID,Check Date,Flight Date,Remove
0,402,2020-02-06,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,185.0,208,2020-04-30,2020-04-30,NaN
1,402,2020-02-07,4,15.0,NaN,NaN,NaN,NaN,11.0,NaN,156.0,208,2020-04-30,2020-04-30,NaN
2,402,2020-02-07,1,16.0,NaN,NaN,NaN,NaN,12.0,NaN,137.0,208,2020-04-30,2020-04-30,NaN
3,402,2020-02-08,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,225.0,208,2020-04-30,2020-07-01,NaN
4,402,2020-02-10,3,15.0,NaN,NaN,NaN,NaN,11.0,NaN,157.0,208,2020-04-30,2020-07-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,738,NaT,4,284.0,12.0,50.0,55.0,0.909091,NaN,NaN,430.0,1172,2020-10-20,NaT,NaN
4855,738,NaT,5,199.0,8.0,45.0,85.0,0.529412,NaN,NaN,358.0,1172,2020-10-20,NaT,NaN
4856,738,NaT,3,280.0,23.0,60.0,80.0,0.750000,NaN,NaN,417.0,1172,2020-10-20,NaT,NaN
4857,738,NaT,4,247.0,10.0,45.0,80.0,0.562500,NaN,NaN,413.0,1172,2020-10-20,NaT,NaN


Add data from weather datasheet

In [55]:
# reading the files
f2 = pd.read_excel(r'weather.xlsx') 
  
# merging the files
f3 = df[["Batch Number","Plant Date","Class","Fresh Weight (g)","Head Weight (g)","Radial Diameter (mm)","Polar Diameter (mm)","Diameter Ratio","Leaves","Density (kg/L)","Leaf Area (cm^2)","Square ID","Check Date","Flight Date","Remove"]].merge(f2[["Flight Date","Solar Radiation [avg]","Precipitation [sum]","Wind Speed [avg]","Wind Speed [max]","Battery Voltage [last]","Leaf Wetness [time]","Air Temperature [avg]","Air Temperature [max]","Air Temperature [min]","Relative Humidity [avg]","Dew Point [avg]","Dew Point [min]","ET0 [result]"]],
on = "Flight Date",how = "left")
  


In [56]:
f3

,Batch Number,Plant Date,Class,Fresh Weight (g),Head Weight (g),Radial Diameter (mm),Polar Diameter (mm),Diameter Ratio,Leaves,Density (kg/L),...,Wind Speed [max],Battery Voltage [last],Leaf Wetness [time],Air Temperature [avg],Air Temperature [max],Air Temperature [min],Relative Humidity [avg],Dew Point [avg],Dew Point [min],ET0 [result]
0,402,2020-02-06,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,7.4,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9
1,402,2020-02-07,4,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,7.4,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9
2,402,2020-02-07,1,16.0,NaN,NaN,NaN,NaN,12.0,NaN,...,7.4,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9
3,402,2020-02-08,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,5.8,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7
4,402,2020-02-10,3,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,5.8,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,738,NaT,4,284.0,12.0,50.0,55.0,0.909091,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4855,738,NaT,5,199.0,8.0,45.0,85.0,0.529412,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856,738,NaT,3,280.0,23.0,60.0,80.0,0.750000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4857,738,NaT,4,247.0,10.0,45.0,80.0,0.562500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Edit Column Name

In [57]:
f3.rename(columns={'Batch Number': 'Batch_No', 'Plant Date': 'Plant_Date','Class': 'class', 'Fresh Weight (g)': 'Fresh_weight','Head Weight (g)': 'Head_Weight', 'Radial Diameter (mm)': 'Radial_diameter','Polar Diameter (mm)': 'Polar_diameter', 'Diameter Ratio': 'Diameter_ratio','Leaves': 'leaves', 'Density (kg/L)': 'Density','Leaf Area (cm^2)': 'Leaf_Area', 'Square ID': 'Square_ID','Check Date': 'Check_Date', 'Flight Date': 'Obervation_date','Remove': 'Remove_row','Solar Radiation [avg]':'Solar_radiation','Precipitation [sum]':'Precipitation','Wind Speed [avg]':'Wind_speed_avg','Wind Speed [max]':'Wind_speed_max','Battery Voltage [last]':'Battery_voltage','Leaf Wetness [time]':'leaf_wetness_time','Air Temperature [avg]':'Air_temperation_avg','Air Temperature [max]':'Air_temperation_max','Air Temperature [min]':'Air_temperation_min','Relative Humidity [avg]':'Relative_humidity_avg','Dew Point [avg]':'Dew_point_avg','Dew Point [min]':'Dew_point_min','ET0 [result]':'AT0_Result'}, inplace=True)
f3.head()

,Batch_No,Plant_Date,class,Fresh_weight,Head_Weight,Radial_diameter,Polar_diameter,Diameter_ratio,leaves,Density,...,Wind_speed_max,Battery_voltage,leaf_wetness_time,Air_temperation_avg,Air_temperation_max,Air_temperation_min,Relative_humidity_avg,Dew_point_avg,Dew_point_min,AT0_Result
0,402,2020-02-06,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,7.4,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9
1,402,2020-02-07,4,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,7.4,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9
2,402,2020-02-07,1,16.0,NaN,NaN,NaN,NaN,12.0,NaN,...,7.4,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9
3,402,2020-02-08,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,5.8,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7
4,402,2020-02-10,3,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,5.8,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7


In [58]:
f3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4859 entries, 0 to 4858
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Batch_No               4859 non-null   int64         
 1   Plant_Date             1821 non-null   datetime64[ns]
 2   class                  4859 non-null   int64         
 3   Fresh_weight           4858 non-null   float64       
 4   Head_Weight            4754 non-null   float64       
 5   Radial_diameter        4754 non-null   float64       
 6   Polar_diameter         4754 non-null   float64       
 7   Diameter_ratio         4752 non-null   float64       
 8   leaves                 100 non-null    float64       
 9   Density                4672 non-null   float64       
 10  Leaf_Area              4857 non-null   float64       
 11  Square_ID              4859 non-null   int64         
 12  Check_Date             4859 non-null   datetime64[ns]
 13  Obe

For expected number of days required for size of lettuce how much produce they can sell them

In [85]:
df=f3

how many days required for crop supply in market for this we add column expected_days by subtracting Obervation_date from Plant_Date 

In [83]:
import datetime

In [84]:
df[['Plant_Date','Obervation_date']] = df[['Plant_Date','Obervation_date']].apply(pd.to_datetime) #if conversion required
df['expected_days'] = (df['Obervation_date'] - df['Plant_Date']).dt.days

In [62]:
df

,Batch_No,Plant_Date,class,Fresh_weight,Head_Weight,Radial_diameter,Polar_diameter,Diameter_ratio,leaves,Density,...,Battery_voltage,leaf_wetness_time,Air_temperation_avg,Air_temperation_max,Air_temperation_min,Relative_humidity_avg,Dew_point_avg,Dew_point_min,AT0_Result,expected_days
0,402,2020-02-06,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9,84.0
1,402,2020-02-07,4,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9,83.0
2,402,2020-02-07,1,16.0,NaN,NaN,NaN,NaN,12.0,NaN,...,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9,83.0
3,402,2020-02-08,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7,144.0
4,402,2020-02-10,3,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,738,NaT,4,284.0,12.0,50.0,55.0,0.909091,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4855,738,NaT,5,199.0,8.0,45.0,85.0,0.529412,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856,738,NaT,3,280.0,23.0,60.0,80.0,0.750000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4857,738,NaT,4,247.0,10.0,45.0,80.0,0.562500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
f3=df

convert Plant Date and Check Date and Obervation_date into integer 
Most regressors need numerical variables

In [64]:
f3["Plant_Date"] = pd.to_datetime(f3["Plant_Date"]).dt.strftime("%Y%m%d")
f3["Check_Date"] = pd.to_datetime(f3["Check_Date"]).dt.strftime("%Y%m%d")
f3["Obervation_date"] = pd.to_datetime(f3["Obervation_date"]).dt.strftime("%Y%m%d")

In [65]:
f3

,Batch_No,Plant_Date,class,Fresh_weight,Head_Weight,Radial_diameter,Polar_diameter,Diameter_ratio,leaves,Density,...,Battery_voltage,leaf_wetness_time,Air_temperation_avg,Air_temperation_max,Air_temperation_min,Relative_humidity_avg,Dew_point_avg,Dew_point_min,AT0_Result,expected_days
0,402,20200206,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9,84.0
1,402,20200207,4,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9,83.0
2,402,20200207,1,16.0,NaN,NaN,NaN,NaN,12.0,NaN,...,6894.0,20.0,9.24,13.07,6.03,89.02,7.3,5.7,1.9,83.0
3,402,20200208,3,15.0,NaN,NaN,NaN,NaN,10.0,NaN,...,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7,144.0
4,402,20200210,3,15.0,NaN,NaN,NaN,NaN,11.0,NaN,...,6884.0,470.0,16.50,20.10,14.02,92.30,15.0,12.8,2.7,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,738,NaN,4,284.0,12.0,50.0,55.0,0.909091,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4855,738,NaN,5,199.0,8.0,45.0,85.0,0.529412,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4856,738,NaN,3,280.0,23.0,60.0,80.0,0.750000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4857,738,NaN,4,247.0,10.0,45.0,80.0,0.562500,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# check missing values in variables# check Null values

In [66]:
f3.isnull().sum()

Batch_No                    0
Plant_Date               3038
class                       0
Fresh_weight                1
Head_Weight               105
Radial_diameter           105
Polar_diameter            105
Diameter_ratio            107
leaves                   4759
Density                   187
Leaf_Area                   2
Square_ID                   0
Check_Date                  0
Obervation_date          4809
Remove_row               4789
Solar_radiation          4809
Precipitation            4809
Wind_speed_avg           4809
Wind_speed_max           4809
Battery_voltage          4809
leaf_wetness_time        4809
Air_temperation_avg      4809
Air_temperation_max      4809
Air_temperation_min      4809
Relative_humidity_avg    4809
Dew_point_avg            4809
Dew_point_min            4809
AT0_Result               4809
expected_days            4809
dtype: int64

#complete missing values with median

In [67]:
f3['Plant_Date'].fillna(f3['Plant_Date'].median(), inplace = True)
f3['Fresh_weight'].fillna(f3['Fresh_weight'].median(), inplace = True)
f3['Head_Weight'].fillna(f3['Head_Weight'].median(), inplace = True)
f3['Radial_diameter'].fillna(f3['Radial_diameter'].median(), inplace = True)
f3['Polar_diameter'].fillna(f3['Polar_diameter'].median(), inplace = True)
f3['Diameter_ratio'].fillna(f3['Diameter_ratio'].median(), inplace = True)
f3['leaves'].fillna(f3['leaves'].mode()[0], inplace = True)
f3['Density'].fillna(f3['Density'].median(), inplace = True)
f3['Leaf_Area'].fillna(f3['Leaf_Area'].median(), inplace = True)

In [68]:
#complete embarked with median
f3['Obervation_date'].fillna(f3['Obervation_date'].median(), inplace = True)

#complete embarked with median
f3['Solar_radiation'].fillna(f3['Solar_radiation'].median(), inplace = True)

#complete missing fare with median
f3['Precipitation'].fillna(f3['Precipitation'].median(), inplace = True)

f3['Wind_speed_avg'].fillna(f3['Wind_speed_avg'].median(), inplace = True)

#complete embarked with median
f3['Wind_speed_max'].fillna(f3['Wind_speed_max'].median(), inplace = True)

#complete missing fare with median
f3['leaf_wetness_time'].fillna(f3['leaf_wetness_time'].median(), inplace = True)

f3['Air_temperation_avg'].fillna(f3['Air_temperation_avg'].median(), inplace = True)

#complete embarked with median
f3['Air_temperation_max'].fillna(f3['Air_temperation_max'].median(), inplace = True)

#complete missing fare with median
f3['Air_temperation_min'].fillna(f3['Air_temperation_min'].median(), inplace = True)

#complete missing fare with median
f3['Relative_humidity_avg'].fillna(f3['Relative_humidity_avg'].median(), inplace = True)

f3['Dew_point_avg'].fillna(f3['Dew_point_avg'].median(), inplace = True)

#complete embarked with median
f3['Dew_point_min'].fillna(f3['Dew_point_min'].median(), inplace = True)

#complete missing fare with median
f3['AT0_Result'].fillna(f3['AT0_Result'].median(), inplace = True)

#complete embarked with median
f3['expected_days'].fillna(f3['expected_days'].median(), inplace = True)

In [69]:
f3.drop('Remove_row', inplace=True, axis=1)
f3.drop('Battery_voltage', inplace=True, axis=1)

In [70]:
f3.isnull().sum()

Batch_No                 0
Plant_Date               0
class                    0
Fresh_weight             0
Head_Weight              0
Radial_diameter          0
Polar_diameter           0
Diameter_ratio           0
leaves                   0
Density                  0
Leaf_Area                0
Square_ID                0
Check_Date               0
Obervation_date          0
Solar_radiation          0
Precipitation            0
Wind_speed_avg           0
Wind_speed_max           0
leaf_wetness_time        0
Air_temperation_avg      0
Air_temperation_max      0
Air_temperation_min      0
Relative_humidity_avg    0
Dew_point_avg            0
Dew_point_min            0
AT0_Result               0
expected_days            0
dtype: int64

In [71]:
f3

,Batch_No,Plant_Date,class,Fresh_weight,Head_Weight,Radial_diameter,Polar_diameter,Diameter_ratio,leaves,Density,...,Wind_speed_max,leaf_wetness_time,Air_temperation_avg,Air_temperation_max,Air_temperation_min,Relative_humidity_avg,Dew_point_avg,Dew_point_min,AT0_Result,expected_days
0,402,20200206,3,15.0,115.0,109.0,100.0,1.023859,10.0,0.216925,...,7.4,20.0,9.24,13.07,6.030,89.02,7.3,5.7,1.9,84.0
1,402,20200207,4,15.0,115.0,109.0,100.0,1.023859,11.0,0.216925,...,7.4,20.0,9.24,13.07,6.030,89.02,7.3,5.7,1.9,83.0
2,402,20200207,1,16.0,115.0,109.0,100.0,1.023859,12.0,0.216925,...,7.4,20.0,9.24,13.07,6.030,89.02,7.3,5.7,1.9,83.0
3,402,20200208,3,15.0,115.0,109.0,100.0,1.023859,10.0,0.216925,...,5.8,470.0,16.50,20.10,14.020,92.30,15.0,12.8,2.7,144.0
4,402,20200210,3,15.0,115.0,109.0,100.0,1.023859,11.0,0.216925,...,5.8,470.0,16.50,20.10,14.020,92.30,15.0,12.8,2.7,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,738,20200527.0,4,284.0,12.0,50.0,55.0,0.909091,8.0,0.216925,...,5.2,0.0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165.0
4855,738,20200527.0,5,199.0,8.0,45.0,85.0,0.529412,8.0,0.216925,...,5.2,0.0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165.0
4856,738,20200527.0,3,280.0,23.0,60.0,80.0,0.750000,8.0,0.216925,...,5.2,0.0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165.0
4857,738,20200527.0,4,247.0,10.0,45.0,80.0,0.562500,8.0,0.216925,...,5.2,0.0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165.0


Save data into file

In [72]:
f3.to_excel("Result.xlsx", index = False)

In [73]:
import pandas as pd
df = pd.read_excel('Result.xlsx')
df

,Batch_No,Plant_Date,class,Fresh_weight,Head_Weight,Radial_diameter,Polar_diameter,Diameter_ratio,leaves,Density,...,Wind_speed_max,leaf_wetness_time,Air_temperation_avg,Air_temperation_max,Air_temperation_min,Relative_humidity_avg,Dew_point_avg,Dew_point_min,AT0_Result,expected_days
0,402,20200206,3,15.0,115,109,100,1.023859,10,0.216925,...,7.4,20,9.24,13.07,6.030,89.02,7.3,5.7,1.9,84
1,402,20200207,4,15.0,115,109,100,1.023859,11,0.216925,...,7.4,20,9.24,13.07,6.030,89.02,7.3,5.7,1.9,83
2,402,20200207,1,16.0,115,109,100,1.023859,12,0.216925,...,7.4,20,9.24,13.07,6.030,89.02,7.3,5.7,1.9,83
3,402,20200208,3,15.0,115,109,100,1.023859,10,0.216925,...,5.8,470,16.50,20.10,14.020,92.30,15.0,12.8,2.7,144
4,402,20200210,3,15.0,115,109,100,1.023859,11,0.216925,...,5.8,470,16.50,20.10,14.020,92.30,15.0,12.8,2.7,142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4854,738,20200527,4,284.0,12,50,55,0.909091,8,0.216925,...,5.2,0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165
4855,738,20200527,5,199.0,8,45,85,0.529412,8,0.216925,...,5.2,0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165
4856,738,20200527,3,280.0,23,60,80,0.750000,8,0.216925,...,5.2,0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165
4857,738,20200527,4,247.0,10,45,80,0.562500,8,0.216925,...,5.2,0,16.50,20.26,11.535,81.61,13.5,10.6,3.3,165


In [74]:
X = df.drop(['Head_Weight','Radial_diameter','Polar_diameter','expected_days'], axis=1)
y = df.iloc[:,[4,5,6,26]]
print(X.shape)
# print(y.head())
print(y.shape)

(4859, 23)
(4859, 4)


In [75]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
model = LinearRegression()
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
model.fit(X_train, y_train)
print("Accuracy: ",model.score(X_val, y_val) * 100)

Accuracy:  79.56038565744652


In [76]:
from sklearn.ensemble import RandomForestRegressor
randomforest = RandomForestRegressor()
randomforest.fit(X_train, y_train)
print("Accuracy: ",randomforest.score(X_val, y_val) * 100)

Accuracy:  94.13223533457302


predict sample head weight radial diameter polar diameter of crop within days

In [77]:
# linear regression for multioutput regression
from sklearn.ensemble import RandomForestRegressor
randomforest = RandomForestRegressor()
# define model
model=randomforest.fit(X_train, y_train)
# make a prediction
row = [402,20200206,3,15,1.023859216,10,0.216924623,185,208,20200430,20200430,159,3.4,3.9,7.4,20,9.24,13.07,6.03,89.02,7.3,5.7,1.9]
yhat = model.predict([row])
# summarize prediction
print(yhat[0])
res=yhat[0]
res=str(res).lstrip('[').rstrip(']')
print(res)

[115.  109.  100.   85.5]
115.  109.  100.   85.5


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Make Test File and Predict 'Head_Weight','Radial_diameter','Polar_diameter' WITH EXPETECED DAYS

In [79]:
X.to_csv("test_agro.csv", index = False)

In [80]:
df = pd.read_csv("test_agro.csv")

In [81]:
from csv import reader
test_list = ['Head_Weight  Radial_diameter  Polar_diameter Expected days']
# skip first line i.e. read header first and then iterate over each row od csv as a list
with open('test_agro.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    csv_reader = reader(read_obj)
    header = next(csv_reader)
    # Check file as empty
    if header != None:
        # Iterate over each row after the header in the csv
        for row in csv_reader:
            list2 = []
            for i in range(len(row[:])):
                list2.append(row[i])
            
            list3 = []
            for j in range(len(list2)):
                t = float(list2[j])
                list3.append(t)
            my_string=model.predict([list3])
            arr = np.array(my_string)
            # Appending String to list
            # using append()
            test_list.append(str(arr).lstrip('[').rstrip(']'))

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

...

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does

In [86]:
for j in test_list:
    print(j)

Head_Weight  Radial_diameter  Polar_diameter Expected days
115.  109.  100.   85.6
115.  109.  100.   85.5
115.   109.   100.   139.82
115.   109.   100.   142.19
115.   109.   100.   142.26
111.64 106.18  97.45 147.6 
115.   109.   100.   141.03
115.  109.  100.  140.9
115.   109.   100.   140.94
115.   109.   100.   142.73
115.   109.   100.   142.08
115.   109.   100.   142.22
115.   109.   100.   140.59
115.   109.   100.   146.26
115.   109.   100.   152.68
115.   109.   100.   147.17
115. 109. 100. 147.
115.   109.   100.   146.62
115.   109.   100.   150.34
115.   109.   100.   150.38
115.   109.   100.   150.42
115.   109.   100.   150.17
115.   109.   100.   161.95
115.   109.   100.   162.05
115.   109.   100.   165.48
115.   109.   100.   164.87
115.   109.   100.   164.96
115.   109.   100.   164.93
115.   109.   100.   164.68
115.  109.  100.  164.5
115.   109.   100.   167.72
115.   109.   100.   174.97
115.   109.   100.   166.71
115.   109.   100.   167.64
115.  109.  1

339.09 127.62 112.16 165.  
316.55 121.88 111.02 165.  
131.41  99.97 105.65 165.  
243.37 124.09 108.71 165.  
242.2  124.19 104.7  165.  
141.76 104.98  99.74 165.  
215.32 124.1  115.41 165.  
 94.44  74.38 108.78 165.  
126.76 102.37  96.88 165.  
 95.18  95.78 104.34 165.  
106.59 104.34  90.97 165.  
 88.23  89.02  80.01 165.  
836.66 156.25 134.76 165.  
844.33 148.27 129.16 165.  
811.92 150.79 120.43 165.  
860.68 151.96 128.95 165.  
841.03 160.09 137.39 165.  
757.12 155.71 132.7  165.  
578.44 149.2  123.28 165.  
590.84 146.8  117.43 165.  
554.8  146.5  119.94 165.  
676.27 141.65 128.1  165.  
505.48 139.06 122.68 165.  
499.64 134.94 112.99 165.  
586.09 140.68 117.94 165.  
541.45 136.62 115.05 165.  
574.05 139.18 111.45 165.  
590.09 146.67 123.09 165.  
485.22 132.25 129.19 165.  
583.94 147.75 126.46 165.  
487.53 142.14 119.15 165.  
349.99 134.35 116.1  165.  
559.94 144.05 125.39 165.  
568.49 160.83 138.48 165.  
582.31 152.06 130.39 165.  
528.29 149.94 137.42

266.9  122.1   93.07 165.  
198.42 114.53  91.22 165.  
215.45 120.35  90.83 165.  
285.4  130.02  95.08 165.  
 98.98  99.23  81.72 165.  
137.4  104.88  87.92 165.  
124.12 100.98  82.63 165.  
125.45 100.1   86.41 165.  
 66.18  82.59  81.36 165.  
126.37 111.13  89.36 165.  
 84.3  100.78  82.44 165.  
126.34 108.31  89.6  165.  
 88.23  96.95  79.2  165.  
 58.79  79.94  70.69 165.  
 33.16  63.68  56.49 165.  
 31.87  64.04  57.37 165.  
 13.66  54.18  49.33 165.  
 36.07  63.96  63.98 165.  
147.99 109.31  83.02 165.  
149.65 103.89  88.1  165.  
148.86 108.38  81.07 165.  
152.84 101.45  82.42 165.  
156.51  97.1   81.23 165.  
169.37 113.71  91.58 165.  
176.16 104.54  89.01 165.  
121.67  92.16  85.49 165.  
160.45 108.74  89.25 165.  
130.61 103.23  86.7  165.  
107.77  99.58  80.64 165.  
108.84  94.4   76.55 165.  
124.59  99.79  89.77 165.  
136.9   97.81  77.9  165.  
106.95 100.07  82.39 165.  
411.2  144.79 111.84 165.  
359.41 144.3  114.11 165.  
291.76 136.44 112.75